In [ ]:
# General libraries
# import jax
import sys


import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from IPython.display import clear_output

# Own code
sys.path.append("../")
from utils.data_utils import create_data, create_dgp_data, transformation, standardize
from utils.tvp_models import TVPVARModel

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

# Set RNG
np.random.seed(12345)

In [15]:
train = 200

M = 7
p = 4
T = train - p
k = M*(M*p+1)

print(f'k: {k}')

k: 203


In [ ]:
ds = pd.read_csv("../data/fred_qd.csv")

In [13]:
transform = True

gdp = transformation(ds["GDPC1"].iloc[2:].to_numpy(), 5, transform)
cpi = transformation(ds["CPIAUCSL"].iloc[2:].to_numpy(), 6, transform)
fedfund = transformation(ds["FEDFUNDS"].iloc[2:].to_numpy(), 2, transform)
compi = transformation(ds["PPIACO"].iloc[2:].to_numpy(), 6, transform)
borrowings = transformation(ds["TOTRESNS"].iloc[2:].to_numpy(), 6, transform)
sp500 = transformation(ds["S&P 500"].iloc[2:].to_numpy(), 5, transform)
m2 = transformation(ds["M2REAL"].iloc[2:].to_numpy(), 5, transform)

# series_total = [gdp, cpi, fedfund]
series_total = [gdp, cpi, fedfund, compi, borrowings , sp500, m2]

In [14]:
standardized_series = standardize(series_total, train)

### TVP-VAR with Variational Bayes (VB)

In [16]:
complete_T = len(gdp)
X_complete, y_complete = create_data(standardized_series,
                                     complete_T,
                                     complete_T-p,
                                     M,
                                     p,
                                     k)

In [17]:
tvp_svss = TVPVARModel(X_complete, y_complete, p, train)
mt1t, St1t = tvp_svss.train(print_status=False)

In [ ]:
h_forecast = 8

mse, alpl = tvp_svss.calculate_metrics(h_forecast)

../utils/tvp_models.py:201: RuntimeWarning: divide by zero encountered in true_divide
  gamma         = 1/(np.multiply(1+(np.divide((1-self.pi0),self.pi0)),np.exp(l_0-l_1)))
../utils/tvp_models.py:201: RuntimeWarning: invalid value encountered in true_divide
  gamma         = 1/(np.multiply(1+(np.divide((1-self.pi0),self.pi0)),np.exp(l_0-l_1)))


In [ ]:
import pickle

dump_to_disk = (mse, alpl)

with open('notebooks/dumps/mse_alpl.pkl', 'wb') as f:
    pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)